In [ ]:
!pip install pyscf

In [2]:
"""
one_shot_dmet_fixed.py

One-shot DMET toy driver — robust AO selection, bath truncation/capping,
iterative bath reduction, and CASCI fallback when FCI is infeasible.

Dependencies:
  pip install pyscf numpy scipy
"""

import numpy as np
from pyscf import gto, scf, ao2mo, fci, mcscf
from scipy.linalg import svd

def one_shot_dmet(mol_geom: str,
                  frag_atoms,
                  basis='sto-3g',
                  charge=0,
                  spin=0,
                  max_fci_orbitals=14,
                  max_bath_cap=8,
                  verbose=True):
    """
    Run a single-shot DMET impurity solve with several robustness measures.

    Parameters
    ----------
    mol_geom : str
        PySCF-style geometry block (XYZ-like multiline string).
    frag_atoms : list[int]
        Atom indices (0-based) forming the fragment.
    basis : str
        Basis set (default 'sto-3g' recommended for debugging; use cc-pVDZ for production).
    max_fci_orbitals : int
        Maximum number of impurity orbitals allowed for attempting full FCI.
    max_bath_cap : int
        Hard cap on bath orbitals (helps control nimp).
    verbose : bool
        Print diagnostics.

    Returns
    -------
    dict
        Result dictionary with energies, impurity size, solver used, etc.
    """

    # Build molecule and SCF
    mol = gto.Mole()
    mol.atom = mol_geom
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.verbose = 0
    mol.build()

    mf = scf.RHF(mol)
    mf.conv_tol = 1e-9
    mf.kernel()

    mo_coeff = mf.mo_coeff
    mo_occ = mf.mo_occ
    nocc = int(np.sum(mo_occ > 0))
    nao = mo_coeff.shape[0]

    # Robust AO selection: use aoslice_by_atom()
    aoslices = mol.aoslice_by_atom()
    frag_ao_indices = []
    for ia in frag_atoms:
        if ia < 0 or ia >= mol.natm:
            raise IndexError(f"Fragment atom index {ia} out of range (0..{mol.natm-1})")
        b0,b1,p0,p1 = aoslices[ia]
        frag_ao_indices.extend(range(p0,p1))
    frag_ao_indices = sorted(set(frag_ao_indices))
    if len(frag_ao_indices) == 0:
        raise RuntimeError("No AOs found for fragment atoms; check atom indices and basis.")

    # Helper function: build impurity AO->imp basis given nbath
    def build_impurity(nbath):
        occ_mo = mo_coeff[:, :nocc]
        P_frag = np.zeros((nao, len(frag_ao_indices)))
        for j, ao_idx in enumerate(frag_ao_indices):
            P_frag[ao_idx, j] = 1.0
        X = P_frag.T @ occ_mo  # (nfragAO x nocc)
        U, svals, Vh = svd(X, full_matrices=False)
        nbath_eff = min(nbath, Vh.shape[0], nocc)
        if nbath_eff <= 0:
            nbath_eff = 1
        bath_mos = occ_mo @ Vh.T[:, :nbath_eff]  # nao x nbath_eff

        frag_aos = np.zeros((nao, len(frag_ao_indices)))
        for j, ao_idx in enumerate(frag_ao_indices):
            frag_aos[ao_idx, j] = 1.0

        impurity_basis_ao = np.hstack([frag_aos, bath_mos])  # nao x (nfrag + nbath)
        S = mol.intor('int1e_ovlp')
        M = impurity_basis_ao.T @ S @ impurity_basis_ao
        eigvals, eigvecs = np.linalg.eigh(M)
        keep = eigvals > 1e-10
        if not np.any(keep):
            raise RuntimeError("Impurity overlap matrix singular.")
        Tmat = eigvecs[:, keep] @ np.diag(1.0/np.sqrt(eigvals[keep]))
        C_imp_ao = impurity_basis_ao @ Tmat   # nao x nimp
        nimp = C_imp_ao.shape[1]
        return C_imp_ao, nbath_eff, nimp, svals

    # Start with a reasonable nbath
    nbath_try = min(len(frag_ao_indices), nocc, max_bath_cap)
    C_imp_ao, nbath_eff, nimp, svals = build_impurity(nbath_try)
    if verbose:
        print(f"Initial fragment AOs = {len(frag_ao_indices)}, nbath = {nbath_eff}, nimp = {nimp}")

    # Iteratively reduce nbath until nimp <= max_fci_orbitals (or nbath==1)
    while nimp > max_fci_orbitals and nbath_try > 1:
        nbath_try -= 1
        C_imp_ao, nbath_eff, nimp, svals = build_impurity(nbath_try)
        if verbose:
            print(f"Reduced nbath -> {nbath_try}: nimp = {nimp}")

    # If still too large, increase singular-value threshold (drop very small singulars)
    if nimp > max_fci_orbitals:
        occ_mo = mo_coeff[:, :nocc]
        P_frag = np.zeros((nao, len(frag_ao_indices)))
        for j, ao_idx in enumerate(frag_ao_indices):
            P_frag[ao_idx, j] = 1.0
        X = P_frag.T @ occ_mo
        U, svals_full, Vh_full = svd(X, full_matrices=False)
        tol = 1e-6
        while nimp > max_fci_orbitals and tol < 1e-1:
            nbath_try = int(np.sum(svals_full > tol))
            nbath_try = min(max(1, nbath_try), max_bath_cap, nocc)
            C_imp_ao, nbath_eff, nimp, svals = build_impurity(nbath_try)
            if verbose:
                print(f"Tolerance {tol:.1e} => nbath {nbath_try}, nimp {nimp}")
            tol *= 10.0

    # Last resort: auto-reduce fragment by dropping last atom(s)
    frag_atoms_current = list(frag_atoms)
    while nimp > max_fci_orbitals and len(frag_atoms_current) > 1:
        dropped = frag_atoms_current.pop()  # drop last atom
        if verbose:
            print(f"Auto-dropping fragment atom {dropped} to reduce impurity")
        frag_ao_indices = []
        for ia in frag_atoms_current:
            b0,b1,p0,p1 = aoslices[ia]
            frag_ao_indices.extend(range(p0,p1))
        frag_ao_indices = sorted(set(frag_ao_indices))
        nbath_try = min(len(frag_ao_indices), nocc, max_bath_cap)
        C_imp_ao, nbath_eff, nimp, svals = build_impurity(nbath_try)
        if verbose:
            print(f"After dropping: fragment AOs {len(frag_ao_indices)}, nbath {nbath_eff}, nimp {nimp}")

    if nimp > max_fci_orbitals and verbose:
        print("Warning: could not reduce nimp below max_fci_orbitals. Will try CASCI fallback.")

    # Build impurity integrals
    hcore_ao = mf.get_hcore()
    h_imp = C_imp_ao.T @ hcore_ao @ C_imp_ao
    eri_ao = mol.intor('int2e')
    eri_imp = ao2mo.incore.general(eri_ao, (C_imp_ao, C_imp_ao, C_imp_ao, C_imp_ao), compact=False)
    eri_imp = eri_imp.reshape((nimp, nimp, nimp, nimp))

    # Electrons on impurity (rounded)
    dm = mf.make_rdm1()
    n_elec_imp = np.trace(C_imp_ao.T @ dm @ C_imp_ao)
    nelec_imp_int = int(round(n_elec_imp))
    if nelec_imp_int % 2 == 0:
        neleca = nelecb = nelec_imp_int // 2
    else:
        neleca = (nelec_imp_int + 1)//2
        nelecb = nelec_imp_int - neleca

    if verbose:
        print("Final fragment AOs:", len(frag_ao_indices))
        print("Number of bath orbitals used:", nbath_eff)
        print("Impurity dimension (nimp):", nimp)
        print("Impurity electron guess (rounded):", nelec_imp_int, "(alpha,beta) =", (neleca, nelecb))

    # Solve impurity: FCI if small enough else CASCI fallback
    E_imp_corr = None
    solver_used = None
    h1 = h_imp
    eri = eri_imp

    if nimp <= max_fci_orbitals:
        try:
            e_fci, ci_vector = fci.direct_spin1.kernel(h1, eri, nimp, (neleca, nelecb), ecore=0.0, verbose=4)
            E_imp_corr = e_fci
            solver_used = 'FCI'
            if verbose:
                print("FCI (impurity) energy (Ha) = {:.12f}".format(e_fci))
        except Exception as e:
            if verbose:
                print("FCI failed:", e)
            E_imp_corr = None

    if E_imp_corr is None:
        # CASCI fallback
        ncas = min(12, max(4, nimp//2))
        nelecas = min(max(2, nelec_imp_int), 2*ncas)
        if nelecas % 2 == 1:
            nelecas -= 1
        if nelecas < 0:
            nelecas = 0
        if verbose:
            print("Using CASCI fallback: ncas =", ncas, ", nelecas =", nelecas)
        try:
            mc = mcscf.CASCI(mf, ncas, nelecas)
            mo0 = C_imp_ao.copy()
            # pad mo0 to have at least same number of columns as mf.mo_coeff
            if mo0.shape[1] < mf.mo_coeff.shape[1]:
                pad_cols = mf.mo_coeff.shape[1] - mo0.shape[1]
                mo0 = np.hstack([mo0, mf.mo_coeff[:, :pad_cols]])
            mc.mo_coeff = mo0
            mc.fcisolver = fci
            e_cas, civec = mc.kernel()
            E_imp_corr = e_cas
            solver_used = 'CASCI'
            if verbose:
                print("CASCI (impurity) energy (Ha) = {:.12f}".format(e_cas))
        except Exception as e:
            raise RuntimeError(f"Both FCI and CASCI impurity solves failed: {e}")

    # Rough DMET energy estimate:
    E_mf = mf.e_tot
    occ_mo = mo_coeff[:, :nocc]
    dm_imp_mf = C_imp_ao.T @ (occ_mo @ occ_mo.T) @ C_imp_ao
    E_imp_mf = np.einsum('ij,ji', h1, dm_imp_mf) + 0.5 * np.einsum('ijkl,li,kj', eri, dm_imp_mf, dm_imp_mf)
    E_env = E_mf - E_imp_mf
    E_total_dmet_approx = E_env + E_imp_corr

    if verbose:
        print("Mean-field total energy (RHF) = {:.12f} Ha".format(E_mf))
        print(f"One-shot DMET approx total energy ({solver_used}) = {E_total_dmet_approx:.12f} Ha")

    return {
        'mol': mol, 'mf': mf,
        'frag_atoms_initial': frag_atoms,
        'frag_ao_indices': frag_ao_indices,
        'nbath': nbath_eff,
        'nimp': nimp,
        'nelec_imp': nelec_imp_int,
        'solver_used': solver_used,
        'E_imp_corr': E_imp_corr,
        'E_total_dmet_approx': E_total_dmet_approx
    }

co2_geom = """
 C    0.000000    0.000000    0.000000
 O    0.000000    0.000000    1.162000
 O    0.000000    0.000000   -1.162000
"""
res = one_shot_dmet(co2_geom, frag_atoms=[0,1], basis='sto-3g', max_fci_orbitals=14, max_bath_cap=6)
print(res)

Initial fragment AOs = 10, nbath = 6, nimp = 15
Reduced nbath -> 5: nimp = 15
Reduced nbath -> 4: nimp = 14
Final fragment AOs: 10
Number of bath orbitals used: 4
Impurity dimension (nimp): 14
Impurity electron guess (rounded): 23 (alpha,beta) = (12, 11)
FCI (impurity) energy (Ha) = -242.179266177228
Mean-field total energy (RHF) = -185.065220119894 Ha
One-shot DMET approx total energy (FCI) = -247.335229810259 Ha
{'mol': <pyscf.gto.mole.Mole object at 0x7f1735abc790>, 'mf': <pyscf.scf.hf.RHF object at 0x7f174efdf810>, 'frag_atoms_initial': [0, 1], 'frag_ao_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'nbath': 4, 'nimp': 14, 'nelec_imp': 23, 'solver_used': 'FCI', 'E_imp_corr': -242.17926617722784, 'E_total_dmet_approx': -247.33522981025916}


In [3]:
nh3_geom = """
N    0.000000    0.000000    0.111100
H    0.000000    0.931600   -0.259200
H    0.806800   -0.465800   -0.259200
H   -0.806800   -0.465800   -0.259200
"""
res = one_shot_dmet(nh3_geom, frag_atoms=[0,1], basis='sto-3g', max_fci_orbitals=14, max_bath_cap=6)
print(res)

Initial fragment AOs = 6, nbath = 5, nimp = 8
Final fragment AOs: 6
Number of bath orbitals used: 5
Impurity dimension (nimp): 8
Impurity electron guess (rounded): 9 (alpha,beta) = (5, 4)
FCI (impurity) energy (Ha) = -67.296207182553
Mean-field total energy (RHF) = -55.452656851781 Ha
One-shot DMET approx total energy (FCI) = -80.474005280125 Ha
{'mol': <pyscf.gto.mole.Mole object at 0x7f1735b1a290>, 'mf': <pyscf.scf.hf.RHF object at 0x7f174efdf690>, 'frag_atoms_initial': [0, 1], 'frag_ao_indices': [0, 1, 2, 3, 4, 5], 'nbath': 5, 'nimp': 8, 'nelec_imp': 9, 'solver_used': 'FCI', 'E_imp_corr': -67.29620718255316, 'E_total_dmet_approx': -80.47400528012511}


In [4]:
nh2cooh_geom = """
C   0.00000000   0.00000000   0.00000000
O   1.20500000   0.00000000   0.00000000
O  -0.74624274   1.12787745   0.00000000
H  -0.11545599   1.85274770   0.00000000
N  -0.78998866  -1.09373606   0.00000000
H  -1.18505755  -0.17595546   0.00000000
H   0.20010645  -1.22531534   0.00000000
"""
res = one_shot_dmet(nh2cooh_geom, frag_atoms=[0,1], basis='sto-3g', max_fci_orbitals=14, max_bath_cap=6)
print(res)

Initial fragment AOs = 10, nbath = 6, nimp = 16
Reduced nbath -> 5: nimp = 15
Reduced nbath -> 4: nimp = 14
Final fragment AOs: 10
Number of bath orbitals used: 4
Impurity dimension (nimp): 14
Impurity electron guess (rounded): 21 (alpha,beta) = (11, 10)
FCI (impurity) energy (Ha) = -260.008229601237
Mean-field total energy (RHF) = -240.180227879533 Ha
One-shot DMET approx total energy (FCI) = -329.648130624360 Ha
{'mol': <pyscf.gto.mole.Mole object at 0x7f1735b094d0>, 'mf': <pyscf.scf.hf.RHF object at 0x7f1735af8e50>, 'frag_atoms_initial': [0, 1], 'frag_ao_indices': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 'nbath': 4, 'nimp': 14, 'nelec_imp': 21, 'solver_used': 'FCI', 'E_imp_corr': -260.00822960123725, 'E_total_dmet_approx': -329.64813062436036}


In [ ]:
import numpy as np
from pyscf import gto, scf

# reuse your 'mol' and 'mf' objects from the DMET run
S = mol.intor('int1e_ovlp')
dm_ao = mf.make_rdm1()   # AO density matrix from RHF
total_elec_via_trace = np.trace(dm_ao @ S)
print("Total electrons (trace(dm @ S)) = ", total_elec_via_trace)
# Also print number of electrons from atomic numbers:
nelec_from_atoms = sum([mol.atom_charge(i) for i in range(mol.natm)])  # sum of Z
print("Total electrons from atom charges (should match above) = ", nelec_from_atoms)

In [ ]:
from pyscf.scf.hf import mulliken_pop_meta_lowdin_ao as mull
pop, charges = mull(mol, dm_ao, verbose=0)   # pop = per-AO Mulliken populations
# frag_ao_indices should be your fragment AO index list
n_elec_on_fragment = pop[frag_ao_indices].sum()
print("Mulliken electrons on fragment (sum of fragment AO pops) =", n_elec_on_fragment)

In [ ]:
I_check = C_imp_ao.T @ S @ C_imp_ao
print("max deviation from identity in C_imp_ao.T @ S @ C_imp_ao =", np.max(np.abs(I_check - np.eye(I_check.shape[0]))))

In [ ]:
from pyscf import mp, cc
# run MP2
mp2 = mp.MP2(mf).run()
print("MP2 total energy = ", mp2.e_tot)
# run CCSD
mycc = cc.CCSD(mf).run()
print("CCSD total energy = ", mycc.e_tot)

In [ ]:
import numpy as np
np.set_printoptions(precision=12, suppress=False)

# 1) Exact impurity electron count (no rounding)
# you used: n_elec_imp = np.trace(C_imp_ao.T @ dm @ C_imp_ao)
dm = mf.make_rdm1()
S = mol.intor('int1e_ovlp')
n_elec_imp_exact = np.trace(C_imp_ao.T @ dm @ C_imp_ao)
print("Impurity electrons (exact float) =", n_elec_imp_exact)
print("Difference total_electrons - n_elec_imp =", np.trace(dm @ S) - n_elec_imp_exact)

# 2) Mulliken populations per atom (full table)
from pyscf.tools import molden
# PySCF's Mulliken helper:
from pyscf import lo
from pyscf.scf import hf
pop, chgs = hf.mulliken_pop_meta_lowdin_ao(mol, dm, verbose=0)
print("Mulliken population per AO (first 40 shown):", pop[:min(len(pop),40)])
# Sum per atom (mol.aoslice_by_atom gives the AO ranges)
aoslices = mol.aoslice_by_atom()
for ia in range(mol.natm):
    b0,b1,p0,p1 = aoslices[ia]
    s = pop[p0:p1].sum()
    print(f"Atom {ia} {mol.atom_pure_symbol(ia):2s} Mulliken electrons = {s:.8f}")

# 3) Check impurity 1RDM in impurity basis and its eigenvalues (natural occupations)
dm_imp_mf = C_imp_ao.T @ dm @ C_imp_ao   # impurity 1RDM (MF projected)
eigvals, eigvecs = np.linalg.eigh(dm_imp_mf)
print("Impurity 1RDM eigenvalues (natural occs) sorted descending:\n", np.sort(eigvals)[::-1])

# 4) Recompute E_imp_mf (mean-field impurity energy) explicitly
h1 = h_imp  # your h_imp = C_imp_ao.T @ hcore_ao @ C_imp_ao
eri = eri_imp  # transformed (nimp x nimp x nimp x nimp)
E_imp_mf = np.einsum('ij,ji', h1, dm_imp_mf) + 0.5 * np.einsum('ijkl,li,kj', eri, dm_imp_mf, dm_imp_mf)
print("E_imp_mf (projected mean-field impurity energy) =", E_imp_mf)

# 5) Reconstruct E_env and the DMET total and compare to your printed numbers
E_mf = mf.e_tot
E_env = E_mf - E_imp_mf
print("E_mf (RHF) =", E_mf)
print("E_env (E_mf - E_imp_mf) =", E_env)

# If you have stored E_imp_corr (from FCI/CASCI), print it (it may be named e_fci or E_imp_corr)
try:
    print("E_imp_corr (from impurity solver) =", E_imp_corr)
    print("Reconstructed E_total_dmet_approx = E_env + E_imp_corr =", E_env + E_imp_corr)
except NameError:
    print("E_imp_corr not found in namespace; print `e_fci` or the variable you used.")

# 6) Sanity bounds: E_total_dmet_approx should be in the neighborhood of correlated full-system energy
# Compare to MP2/CCSD that you computed:
print("Full-system MP2 =", mp2.e_tot)
print("Full-system CCSD =", mycc.e_tot)


In [ ]:
!pip install qiskit_algorithms

In [ ]:
%pip install -U qiskit
%pip install -U qiskit-ibm-runtime
%pip install -U qiskit-nature
%pip install -U qiskit-algorithms
%pip install -U qiskit-addon-sqd
%pip install -U ffsim
%pip install -U pyscf

In [ ]:
%pip install qiskit==1.4.2

In [1]:
import qiskit
# Import common packages first
import numpy as np
from math import comb
import warnings
import pyscf
import matplotlib.pyplot as plt
import pickle
from functools import partial
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.drivers import GaussianForcesDriver
from qiskit_nature.second_q.mappers import DirectMapper
from qiskit_nature.second_q.problems import HarmonicBasis

In [9]:
from qiskit_algorithms import NumPyMinimumEigensolver
import matplotlib.pyplot as plt
def modelMolec(atomGeo, use_vqe=False):
    driver = PySCFDriver(
        atom=atomGeo,
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )

    es_problem = driver.run()

    transformer2 = ActiveSpaceTransformer(num_electrons=4, num_spatial_orbitals=4)
    es_problem = transformer2.transform(es_problem)

    mapper = JordanWignerMapper()

    solver = NumPyMinimumEigensolver() if not use_vqe else VQE(
        Estimator(),
        UCCSD(
            es_problem.num_spatial_orbitals,
            es_problem.num_particles,
            mapper,
            initial_state=HartreeFock(
                es_problem.num_spatial_orbitals,
                es_problem.num_particles,
                mapper,
            ),
        ),
        SLSQP()
    )

    calc = GroundStateEigensolver(mapper, solver)
    res = calc.solve(es_problem)
    return res
co2_geom = """
 C    0.000000    0.000000    0.000000
 O    0.000000    0.000000    1.162000
 O    0.000000    0.000000   -1.162000
"""
co2_eneg = modelMolec(
    "C    0.000000    0.000000    0.000000; "
    "O    0.000000    0.000000    1.162000; "
    "O    0.000000    0.000000   -1.162000; "
)
print(f"CO2 Energy: {co2_eneg}")

CO2 Energy: === GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -243.429141950042
  - computed part:      -4.719757993728
  - ActiveSpaceTransformer extracted energy part: -238.709383956314
~ Nuclear repulsion energy (Hartree): 58.291465574664
> Total ground state energy (Hartree): -185.137676375378
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  0.0]
    - computed part:      [0.0  0.0  0.0]
    - ActiveSpaceTransformer extracted energy part: [0.0  0.0  0.0]
  > Dipole moment (a.u.): [0.0  0.0  0.0]  Total: 0.0
                 (debye): [0.0  0.0  0.0]  Total: 0.0
 


In [10]:
nh3_geom = """
N    0.000000    0.000000    0.111100
H    0.000000    0.931600   -0.259200
H    0.806800   -0.465800   -0.259200
H   -0.806800   -0.465800   -0.259200
"""
nh3_eneg = modelMolec(
    "N    0.000000    0.000000    0.111100; "
    "H    0.000000    0.931600   -0.259200; "
    "H    0.806800   -0.465800   -0.259200; "
    "H   -0.806800   -0.465800   -0.259200; "
    
)
print(f"NH3 Energy: {nh3_eneg}")

NH3 Energy: === GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -67.526101382989
  - computed part:      -5.350425032998
  - ActiveSpaceTransformer extracted energy part: -62.175676349991
~ Nuclear repulsion energy (Hartree): 12.068827248365
> Total ground state energy (Hartree): -55.457274134624
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  0.00018897]
 
  0: 
  * Electronic dipole moment (a.u.): [-0.000000000207  -0.009299098795  0.692408940743]
    - computed part:      [0.0  0.798639585179  0.859314783555]
    - ActiveSpaceTransformer extracted energy part: [-0.000000000207  -0.807938683975  -0.166905842812]
  > Dipole moment (a.u.): [0.000000000207  0.009299098795  -0.692219970743]  Total: 0.692282428734
                 (debye): [0.000000000526  0.023635949315  -1.759447501696]  Total: 1.759606254059
 


In [11]:
nh2cooh_geom = """
C   0.00000000   0.00000000   0.00000000
O   1.20500000   0.00000000   0.00000000
O  -0.74624274   1.12787745   0.00000000
H  -0.11545599   1.85274770   0.00000000
N  -0.78998866  -1.09373606   0.00000000
H  -1.18505755  -0.17595546   0.00000000
H   0.20010645  -1.22531534   0.00000000
"""
nh2cooh_eneg = modelMolec(
    "C   0.00000000   0.00000000   0.00000000; "
    "O   1.20500000   0.00000000   0.00000000; "
    "O  -0.74624274   1.12787745   0.00000000; "
    "H  -0.11545599   1.85274770   0.00000000; "
    "N  -0.78998866  -1.09373606   0.00000000; "
    "H  -1.18505755  -0.17595546   0.00000000; "
    "H   0.20010645  -1.22531534   0.00000000; "
)
print(f"NH2COOH Energy: {nh2cooh_eneg}")

NH2COOH Energy: === GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -369.639679428588
  - computed part:      -4.139599003408
  - ActiveSpaceTransformer extracted energy part: -365.500080425181
~ Nuclear repulsion energy (Hartree): 129.438893116141
> Total ground state energy (Hartree): -240.200786312447
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 4.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [-5.59409886  3.43617231  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [-5.526380125099  2.691518817392  0.0]
    - computed part:      [2.165334991046  -2.808063202859  0.0]
    - ActiveSpaceTransformer extracted energy part: [-7.691715116146  5.49958202025  0.0]
  > Dipole moment (a.u.): [-0.067718734901  0.744653492608  0.0]  Total: 0.747726320996
                 (debye): [-0.172123839205  1.892720208279  0.0]  Total: 1.900530558253
 
